#### kitt_net neural network training

In [1]:
from kitt_net import FeedForwardNet
from shelve import open as open_shelve
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib import pyplot as plt
%matplotlib qt
from numpy import newaxis

In [2]:
net = FeedForwardNet(hidden=[50], tf_name='Sigmoid')

In [3]:
dataset = open_shelve('../examples/speech/dataset_speech_500_s+z.ds')

In [4]:
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'], 
        learning_rate=0.07, n_epoch=100, c_stable=20, req_acc=1.0, batch_size=1)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 440
	% number of classes: 3
	% class labels: [u'%', u's', u'z']
	% net structure: [440, 50, 3]
	% net transfer function: Sigmoid

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 440
	% number of training samples: 16000
	% number of validation samples: 2000
	% learning rate: 0.07
	% mini-batch size: 1
	% maximum number of epochs (t.c.): 100
	% maximum number of stable epochs (t.c.): 20
	% required accuracy (t.c.): 1.0
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  0.95/0.0151		0.95/0.0162		5.5821 s
 2	  0.95/0.0146		0.95/0.0153		5.0195 s
 3	  0.95/0.0143		0.95/0.0148		4.8321 s
 4	  0.95/0.0139		0.95/0.0151		4.9171 s
 5	  0.95/0.0136		0.95/0.0150		4.8402 s
 6	  0.95/0.0135

KeyboardInterrupt: 

In [6]:
net.prune(req_acc=0.55, req_err=0.05, n_epoch=5, 
          levels=(75, 50, 30, 20, 10, 7, 5, 3, 2, 1, 0), verbose_learning=True)


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [440, 50, 40]
	% net initial number of synapses (w, b): (24000, 90)
	% min required accuracy (s.c.): 0.55
	% max required error (s.c.): 0.05
	% maximum number of re-training epochs (gu.c.): 5
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [440, 50, 40]       (24000, 90)            None           75             None  

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 440
	% number of training samples: 16000
	% number of validation samples: 2000
	% learning rate: 0.07
	% mini-batch size: 1
	% maximum number of epochs (t.c.): 5
	% maximum number of stable 

KeyboardInterrupt: 

In [8]:
print 'Net structure to be dumped:', net.structure, '| Number of synapses:', net.count_synapses()
net.dump('../examples/speech/net_speech_500_pruned.net')

Net structure to be dumped: [377, 50, 40] | Number of synapses: (1353, 90)

--------------------------------------------------------------------
-- Net dumped as ../examples/speech/net_speech_500_pruned.net


In [9]:
net.v_data = net.prepare_data(x=dataset['x_val'], y=dataset['y_val'])
net.t_data = net.prepare_data(x=dataset['x'], y=dataset['y'])
test_data = net.prepare_data(x=dataset['x_test'], y=dataset['y_test'])

In [10]:
predictions = [net.predict(x)[0][0] for x, y in test_data]
cm = confusion_matrix(y_true=dataset['y_test'], y_pred=predictions, labels=net.labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, newaxis]
labels = [label for label in net.labels if label in dataset['y_test']]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()

print accuracy_score(y_true=dataset['y_test'], y_pred=predictions)
print cm

0.403903903904
[[ 0.3   0.    0.   ...,  0.    0.    0.  ]
 [ 0.    0.44  0.02 ...,  0.    0.    0.  ]
 [ 0.    0.    0.42 ...,  0.    0.    0.04]
 ..., 
 [ 0.02  0.    0.   ...,  0.34  0.    0.02]
 [ 0.    0.    0.   ...,  0.    0.4   0.  ]
 [ 0.    0.    0.02 ...,  0.    0.    0.5 ]]
